In [18]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

## Helper function

In [19]:
import requests
import base58, base64
import base64
def base64url_decode(input):
    rem = len(input) % 4

    if rem > 0:
        input += b'=' * (4 - rem)

    return base64.urlsafe_b64decode(input)

#base64 url encode
def base64url_encode(input):
    return base64.urlsafe_b64encode(input).rstrip(b'=')


def base58_to_jwk(base58secret, crv):
    """
    Converts a base58-encoded secret to a JWK dict.
    :param base58secret: base58-encoded secret
    :return: JWK dict
    """
    hex_key = base58.b58decode(base58secret)
    base64d = base64url_encode(hex_key).decode()
    jwk = {
        "kty": "OKP",
        "crv": crv,
        "x": base64d,
        'kid': uuid.uuid4().hex
    }
    return jwk

from pprint import pprint
def resolve_did_web(did: str) -> DIDDoc:
    """Resolve DIDDoc from did:web"""
    #remove did:web: from did
    did = did[8:]
    #split on : and join with /
    path = "https://"+ "/".join(did.split(":")) +'/.well-known/did.json'
    did_doc_json = requests.get(path).json()
    # pprint(did_doc_json,)
    a = DIDDoc(
        did=did_doc_json['id'],
        key_agreement_kids=['did:web:verifiable.ink#1'],
        authentication_kids=['did:web:verifiable.ink#0'],
        verification_methods=[
        VerificationMethod(
            id=did_doc_json['verificationMethod'][0]['id'],
            type=VerificationMethodType.JSON_WEB_KEY_2020,
            controller=did_doc_json['verificationMethod'][0]['controller'],
            verification_material=VerificationMaterial(
                format=VerificationMaterialFormat.JWK,
                
                value=json.dumps(base58_to_jwk('3zSE11h82KtPYPj8p9cTgzr6yDWFYEsfM19xc1K5vjKY','Ed25519'))
            )
        ),
        VerificationMethod(
            id=did_doc_json['verificationMethod'][1]['id'],
            type=VerificationMethodType.JSON_WEB_KEY_2020,
            controller=did_doc_json['verificationMethod'][1]['controller'],
            verification_material=VerificationMaterial(
                format=VerificationMaterialFormat.JWK,
                value=json.dumps(base58_to_jwk('3zSE11h82KtPYPj8p9cTgzr6yDWFYEsfM19xc1K5vjKY','X25519'))
            )
        )],
        didcomm_services=[DIDCommService(
            id=did_doc_json['service'][0]['id'],
            service_endpoint=did_doc_json['service'][0]['serviceEndpoint'],
            routing_keys=[],
            accept=did_doc_json['service'][0]['accept']
        )]
        
    )

    return a
doc = resolve_did_web('did:web:verifiable.ink')


In [20]:
# import requests,os

# url= 'https://purl.imsglobal.org/spec/ob/v3p0/context.json'
# context_name='test-context'

# #get context text
# ctx = requests.get('https://purl.imsglobal.org/spec/ob/v3p0/context.json').content
# #make it node friendly
# ctx_new = 'module.exports=' + ctx.decode('utf-8') 
# #write the context to the file
# cwd = os.getcwd()
# with open(f'/Users/alex/Projects/rootid/rootswallet/node_modules/@sphereon/rn-vc-js/lib/contexts/'+context_name+'.js', 'w') as f:
#     f.write(ctx_new)
# #read /Users/alex/Projects/rootid/rootswallet/node_modules/@sphereon/rn-vc-js/lib/contexts/index.js into an array
# with open(f'/Users/alex/Projects/rootid/rootswallet/node_modules/@sphereon/rn-vc-js/lib/contexts/index.js') as f:
#     lines = f.readlines()
# #find all the index of the lines that start with 'https'
# indexes = [i for i, s in enumerate(lines) if 'https' in s]
# #find the line with the last context mapping
# latest = indexes.pop()
# #add context mapping line so the file is downloaded is rechable to the documentLoader
# lines.insert(latest+1,f'  \'{url}\': require(\'./{context_name}\'),\n')
# #write the new lines to the file
# with open('/Users/alex/Projects/rootid/rootswallet/node_modules/@sphereon/rn-vc-js/lib/contexts/index.js', 'w') as f:
#     f.writelines(lines)

In [21]:
# import os 
# cwd = os.getcwd()
# print(cwd)

/Users/alex/Projects/rootid/didcomm-mediator/sample-notebooks


In [22]:
secrets_resolver = SecretsResolverDemo()

In [23]:


class DidUniversalResolver(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        if did.startswith("did:peer"):
            did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
            did_doc = DIDDocPeerDID.from_json(did_doc_json)

            return DIDDoc(
                did=did_doc.did,
                key_agreement_kids = did_doc.agreement_kids,
                authentication_kids = did_doc.auth_kids,
                verification_methods = [
                    VerificationMethod(
                        id = m.id,
                        type = VerificationMethodType.JSON_WEB_KEY_2020,
                        controller = m.controller,
                        verification_material = VerificationMaterial(
                            format = VerificationMaterialFormat.JWK,
                            value = json.dumps(m.ver_material.value)
                        )
                    )
                    for m in did_doc.authentication + did_doc.key_agreement
                ],
                didcomm_services = [
                    DIDCommService(
                        id = s.id,
                        service_endpoint = s.service_endpoint,
                        routing_keys = s.routing_keys,
                        accept = s.accept
                    )
                    for s in did_doc.service
                    if isinstance(s, DIDCommServicePeerDID)
                ] if did_doc.service else []
            )
        else:
            return resolve_did_web(did)


In [24]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )
        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did, auth_keys[0]


In [25]:
_tmpp_did_holder, _tmpp_keys_holder_jwk  = await create_peer_did( 1,1, service_endpoint="https://example.com")


In [39]:
_tmpp_keys_holder_jwk

({'crv': 'Ed25519',
  'x': 'vUNE4zA7G6GiAbSOl591o_WrWqbvKTJa4eSLWzjoPjk',
  'd': 'kpjhLmPnFGBXA6N-_cpZqSXpcT0ER0ZQlP_8sDW7fnk',
  'kty': 'OKP',
  'kid': 'did:peer:2.Ez6LSnRkrkowwK1DNVngsPGknLoA1AW8GTMv9FP1erZXtPSxu.Vz6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl19#6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz'},
 {'crv': 'Ed25519',
  'x': 'vUNE4zA7G6GiAbSOl591o_WrWqbvKTJa4eSLWzjoPjk',
  'kty': 'OKP',
  'kid': '9eJYENR8_SJF_1cV8HWfeqJTB4t6FJakbFc1DkT4wJI'})

In [37]:
_tmp_did_doc = json.loads(peer_did.resolve_peer_did(_tmpp_did_holder))
_tmp_did_doc

{'id': 'did:peer:2.Ez6LSnRkrkowwK1DNVngsPGknLoA1AW8GTMv9FP1erZXtPSxu.Vz6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl19',
 'authentication': [{'id': 'did:peer:2.Ez6LSnRkrkowwK1DNVngsPGknLoA1AW8GTMv9FP1erZXtPSxu.Vz6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl19#6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz',
   'type': 'Ed25519VerificationKey2020',
   'controller': 'did:peer:2.Ez6LSnRkrkowwK1DNVngsPGknLoA1AW8GTMv9FP1erZXtPSxu.Vz6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl19',
   'publicKeyMultibase': 'z6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz'}],
 'keyAgreement': [{'id': 'did:peer:2.Ez6LSnRkrkowwK1DNVngsPGknLoA1AW8GTMv9FP1erZXtPSxu.Vz6MksC4bNj7jLmjha8VyLnfgnpqaAL5AzN5HkGgRYr3SJhdz.SeyJpZCI6Im5ldy1pZCI

In [26]:
did_holder, keys_holder_jwk  = await create_peer_did( 1,1, service_endpoint="https://example.com", service_routing_keys=[_tmpp_did_holder])
print("did_holder", did_holder)

did_holder did:peer:2.Ez6LSgYTCJbiHEYTvxGm28hRLfYEthdEg4Z6xkUu9ytQRqTGr.Vz6MkrcDP4f9ddywn1nmVBKk3RwYX4kpHwjhjgz49KrQCSBmj.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU25Sa3Jrb3d3SzFETlZuZ3NQR2tuTG9BMUFXOEdUTXY5RlAxZXJaWHRQU3h1LlZ6Nk1rc0M0Yk5qN2pMbWpoYThWeUxuZmducHFhQUw1QXpONUhrR2dSWXIzU0poZHouU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyVjRZVzF3YkdVdVkyOXRJaXdpWVNJNld5SmthV1JqYjIxdEwzWXlJbDE5Il19


In [27]:
#resolve did peer
did_doc_holder = await DidUniversalResolver().resolve(did_holder)
print("did_doc_holder", did_doc_holder)

did_doc_holder DIDDoc(did='did:peer:2.Ez6LSgYTCJbiHEYTvxGm28hRLfYEthdEg4Z6xkUu9ytQRqTGr.Vz6MkrcDP4f9ddywn1nmVBKk3RwYX4kpHwjhjgz49KrQCSBmj.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU25Sa3Jrb3d3SzFETlZuZ3NQR2tuTG9BMUFXOEdUTXY5RlAxZXJaWHRQU3h1LlZ6Nk1rc0M0Yk5qN2pMbWpoYThWeUxuZmducHFhQUw1QXpONUhrR2dSWXIzU0poZHouU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyVjRZVzF3YkdVdVkyOXRJaXdpWVNJNld5SmthV1JqYjIxdEwzWXlJbDE5Il19', key_agreement_kids=['did:peer:2.Ez6LSgYTCJbiHEYTvxGm28hRLfYEthdEg4Z6xkUu9ytQRqTGr.Vz6MkrcDP4f9ddywn1nmVBKk3RwYX4kpHwjhjgz49KrQCSBmj.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU25Sa3Jrb3d3SzFETlZuZ3NQR2tuTG9BMUFXOEdUTXY5RlAxZXJaWHRQU3h1LlZ6Nk1rc0M0Yk5qN2pMbWpoYThWeUxuZmducHFhQUw1QXpONUhrR2dSWXIzU0poZHouU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyVjRZVzF3YkdVdVkyOXRJaXdpWVNJNld5S

In [28]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(did_holder))
issuer_did_doc

{'id': 'did:peer:2.Ez6LSgYTCJbiHEYTvxGm28hRLfYEthdEg4Z6xkUu9ytQRqTGr.Vz6MkrcDP4f9ddywn1nmVBKk3RwYX4kpHwjhjgz49KrQCSBmj.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU25Sa3Jrb3d3SzFETlZuZ3NQR2tuTG9BMUFXOEdUTXY5RlAxZXJaWHRQU3h1LlZ6Nk1rc0M0Yk5qN2pMbWpoYThWeUxuZmducHFhQUw1QXpONUhrR2dSWXIzU0poZHouU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyVjRZVzF3YkdVdVkyOXRJaXdpWVNJNld5SmthV1JqYjIxdEwzWXlJbDE5Il19',
 'authentication': [{'id': 'did:peer:2.Ez6LSgYTCJbiHEYTvxGm28hRLfYEthdEg4Z6xkUu9ytQRqTGr.Vz6MkrcDP4f9ddywn1nmVBKk3RwYX4kpHwjhjgz49KrQCSBmj.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL2V4YW1wbGUuY29tIiwiYSI6WyJkaWRjb21tL3YyIl0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU25Sa3Jrb3d3SzFETlZuZ3NQR2tuTG9BMUFXOEdUTXY5RlAxZXJaWHRQU3h1LlZ6Nk1rc0M0Yk5qN2pMbWpoYThWeUxuZmducHFhQUw1QXpONUhrR2dSWXIzU0poZHouU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwyVjRZVzF3YkdVdVkyOXRJaXdpWVNJNld5SmthV1JqYjIxd

In [29]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    # pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/propose-credential",
    frm = did_holder,
    to = ['did:web:verifiable.ink'],
    body= {}      
)

holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DidUniversalResolver()
    ),
    message = holder_response_message,
    frm = did_holder,
    to = 'did:web:verifiable.ink',
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False,forward=False)
)

{'epk': <authlib.jose.rfc8037.okp_key.OKPKey object at 0x11b2fbeb0>, 'header': {'epk': {'crv': 'X25519', 'x': 'RnEowZvxhtuZrEfKLg4Ll8-Po_j7zerEkCCZj38gLC0', 'kty': 'OKP'}}, 'cek': b'E\xc9\xff\xf3\xe0\xe6\x04\x86F\x06\x84T\xc5\xdc\xf2kC\x99\xe1C7\xfb\xd8\xf8\x1a/\x06W\x12a\\\xa6\xc9\xfe\xa5\x88\xef\xff\x84wX\xd6\x0f\xb0\xa1\xb8\x964\x9d\xd8\xb5\x05\x08\xf4\x15\xba\xe9\xf36Q\xb4\xc4\xe5\x0e'}


In [40]:
json.loads(holder_packed_msg.packed_msg)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post('https://verifiable.ink/didcomm/', headers=headers, json =json.loads(holder_packed_msg.packed_msg))
print(resp)



<Response [400]>


In [31]:
json.loads(holder_packed_msg.packed_msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTJkWlZFTktZbWxJUlZsVWRuaEhiVEk0YUZKTVpsbEZkR2hrUldjMFdqWjRhMVYxT1hsMFVWSnhWRWR5TGxaNk5rMXJjbU5FVURSbU9XUmtlWGR1TVc1dFZrSkxhek5TZDFsWU5HdHdTSGRxYUdwbmVqUTVTM0pSUTFOQ2JXb3VVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3eVZqUlpWekYzWWtkVmRWa3lPWFJKYVhkcFdWTkpObGQ1U210aFYxSnFZakl4ZEV3eldYbEpiREJ6U1c1SmFVOXNjMmxhUjJ4clQyNUNiRnBZU1RaTmFUVkdaV3BhVFZVeU5WTmhNMHB5WWpOa00xTjZSa1ZVYkZwMVdqTk9VVkl5ZEhWVVJ6bENUVlZHV0U5RlpGVlVXRmsxVW14QmVGcFlTbUZYU0ZKUlZUTm9NVXhzV2paT2F6RnlZekJOTUZsck5YRk9NbkJOWWxkd2IxbFVhRmRsVlhoMVdtMWtkV05JUm1oUlZYY3hVVmh3VDA1VmFISlNNbVJUVjFoSmVsVXdjRzlhU0c5MVZUSldOVk51UW1GUk1Hc3lVMWN3TVdKSFVqVk5XRUpoVVRCc2VsTlhOVkpoVlRsd1UyMTBhVlV3YkhwVFZ6Vk9ZVlU1Y0ZOdE9XdFRSa296V1ROd2RtUnJkM2xXYWxKYVZucEdNMWxyWkZaa1ZtdDVUMWhTU21GWVpIQlhWazVLVG14a05WTnRkR2hXTVVweFdXcEplR1JGZDNwWFdHeEtZa

### Reading issuer OOB message

In [98]:
# oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/invitation_url").text
print(oob_url)

http://127.0.0.1:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiYmJhNTY1N2QtZjNjOC00ZWU3LWIxMGEtMjU3ZmM2OTVmMDI5IiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNvcmZ5SlBYcHNFQWJ2Z1Z6eVBQZ3hvYllSQUtydWk2Q1l1S3p2QzJYRWR4Ny5WejZNa3BmWHhCc3B1WnBCQWRyQ2VXTFc0a2t3aEFmeThyWHdQVjhRQzQzdERlb1dILlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoic3RyZWFtbGluZWQtdmMiLCJhY2NlcHQiOlsiZGlkY29tbS92MiJdLCJpc3N1ZXJfa2V5IjoiZGlkOmtleTp6Nk1rcURWWnZBaTd4ekJIYzVKcEFxREFzQ2JQd0Vra0hXeHZjbkdMaG5WMlZKZlMifX0


In [99]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': 'bba5657d-f3c8-4ee7-b10a-257fc695f029', 'from': 'did:peer:2.Ez6LSorfyJPXpsEAbvgVzyPPgxobYRAKrui6CYuKzvC2XEdx7.Vz6MkpfXxBspuZpBAdrCeWLW4kkwhAfy8rXwPV8QC43tDeoWH.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'streamlined-vc', 'accept': ['didcomm/v2'], 'issuer_key': 'did:key:z6MkqDVZvAi7xzBHc5JpAqDAsCbPwEkkHWxvcnGLhnV2VJfS'}}


In [150]:
import didkit
def generate_ed25519_key():
    """ Generate Ed25519 key """
    jwk = didkit.generate_ed25519_key()
    did = didkit.key_to_did("key", jwk)
    return did, jwk
    
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LSr5LPJGmUgmrGqvycAgqfeWDTp3o6ki9DqKAWZsKS5eBB.Vz6MkpfFYUQ2s2RrUtNBVAN3TBri53P8c5UUtLyNsgkmMaVMv.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


In [122]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    # pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/propose-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body= {}      
)

holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, json =holder_packed_msg.packed_msg)
print(resp)




http://127.0.0.1:8000
<Response [202]>


In [124]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(json.dumps(credential_unpack_msg.message.attachments[0].data.json, indent=4))

{
    "options": {
        "challenge": "c13abd0fe67fcd1f3513e32b1ed5ad7c721e598ea0d1b9334ffecaa8e1a2fce6",
        "domain": "roots.id"
    },
    "credential_manifest": {
        "id": "c001006f847f39073113b0ed862ff88c3c49669b4bbf5a30fcbec9f62d35cf98",
        "version": "0.1.0",
        "issuer": {
            "id": "did:key:z6MkwDWXN87VCMUVwR4GuAFSpUyHuCpJD34GWF85Dhyne7rL",
            "name": "Roots ID Issuer"
        },
        "format": {
            "ldp_vc": {
                "proof_type": [
                    "Ed25519Signature2018"
                ]
            }
        },
        "output_descriptors": [
            {
                "id": "c00200131bb229a058775ebe3134286b577210e7a90d2b44d29d2f603b744bdd",
                "schema": "https://purl.imsglobal.org/spec/ob/v3p0/vocab.html#OpenBadgeCredential"
            }
        ],
        "presentation_definition": {
            "id": "c00400f85e0aae79eb62b099a4aab4eb54cc99df6ca444b83a6fb04c580c3380",
            "input_descri

## Prepare request-credential

The holder needs to create a did:peer to communicate with the issuer:

In [103]:
holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LSff5cxwvPXYvfP9KxtXx3tuikxBt9yRzZcfWKskPFyzEs.Vz6MkmFC5xTHkGEnjx9sK8Pz1fz5kvwjyrsV9QmaVKcNXQSFr.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


The following is a JSON-LD credential request that the holder will submmit to the issuer.

In [104]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-vc-edu-plugfest-1-context.json"
        ],
        "id": str(uuid.uuid4()),
        "type": ["VerifiableCredential", "OpenBadgeCredential"],
        "issuer": {'id': received_msg_decoded['body']['issuer_key']},
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
          "credentialSubject": {
            "type": "AchievementSubject",
            "id": holder_did,
            "achievement": {
              "type": "Achievement",
              "name": "IIW 2022 DEMO",
              "description": "This wallet can display this Open Badge 3.0",
              "criteria": {
                "type": "Criteria",
                "narrative": "The first cohort of the JFF Plugfest 1 in May/June of 2021 collaborated to push interoperability of VCs in education forward."
              },
              "image": "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/images/plugfest-1-badge-image.png"
            }
          },
        "options": {
            "proofType": "Ed25519Signature2018"
        }
    }
}

In [111]:
cred = {'credential':{
    "@context": [
        "https://www.w3.org/2018/credentials/v1",
        # "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-vc-edu-plugfest-1-context.json"
    ],
    "id": "http://example.org/credentials/3731",
    "type": ["VerifiableCredential"],
    "issuer": received_msg_decoded['body']['issuer_key'],
    "issuanceDate": "2020-08-19T21:41:50Z",
    "credentialSubject": {
        "id": "did:example:d23dd687a7dc6787646f2eb98d0",
    },

        "options": {
            "proofType": "Ed25519Signature2018"
        }
}

}

Finally the holder prepares the didcomm message following issue-credential/3.0 protocol:

In [112]:

holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=cred)
                )
    ]
                        
)

In [113]:
holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

In [114]:

issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, json = json.loads(holder_packed_msg.packed_msg))
print(resp)


http://127.0.0.1:8000
<Response [500]>


### Issued Verifiable Credential Received

In [109]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [246]:
print(credential_unpack_msg)

UnpackResult(message=Message(id='b521167b-d9b5-48c7-b377-0cbf80240581', type='https://didcomm.org/issue-credential/3.0/issue-credential', body={'goal_code': 'issue-credential', 'comment': 'some comment'}, frm=None, to=None, created_time=None, expires_time=None, from_prior=None, please_ack=None, ack=None, thid=None, pthid=None, attachments=[Attachment(data=AttachmentDataJson(json='{"@context":"https://www.w3.org/2018/credentials/v1","id":"http://example.org/credentials/3731","type":["VerifiableCredential"],"credentialSubject":{"id":"did:example:d23dd687a7dc6787646f2eb98d0"},"issuer":"did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD","issuanceDate":"2020-08-19T21:41:50Z","proof":{"type":"Ed25519Signature2018","proofPurpose":"assertionMethod","verificationMethod":"did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD#z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD","created":"2022-11-02T18:46:30.247Z","jws":"eyJhbGciOiJFZERTQSIsImNyaXQiOlsiYjY0Il0sImI2NCI6ZmFsc2V9..KvNTOJ8eL_0

In [247]:
print(json.dumps(credential_unpack_msg.message.attachments[0].data.json, indent=4))

"{\"@context\":\"https://www.w3.org/2018/credentials/v1\",\"id\":\"http://example.org/credentials/3731\",\"type\":[\"VerifiableCredential\"],\"credentialSubject\":{\"id\":\"did:example:d23dd687a7dc6787646f2eb98d0\"},\"issuer\":\"did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD\",\"issuanceDate\":\"2020-08-19T21:41:50Z\",\"proof\":{\"type\":\"Ed25519Signature2018\",\"proofPurpose\":\"assertionMethod\",\"verificationMethod\":\"did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD#z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD\",\"created\":\"2022-11-02T18:46:30.247Z\",\"jws\":\"eyJhbGciOiJFZERTQSIsImNyaXQiOlsiYjY0Il0sImI2NCI6ZmFsc2V9..KvNTOJ8eL_0lZ_a8xcEOTXpMiqH6aaMz5tsRM5t3B7QrAT6Hq3sVELCy7RGicpgRhsdDippMLFZUrhVXWRAtDw\"}}"


In [250]:
json.loads(credential_unpack_msg.message.attachments[0].data.json)

{'@context': 'https://www.w3.org/2018/credentials/v1',
 'id': 'http://example.org/credentials/3731',
 'type': ['VerifiableCredential'],
 'credentialSubject': {'id': 'did:example:d23dd687a7dc6787646f2eb98d0'},
 'issuer': 'did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD',
 'issuanceDate': '2020-08-19T21:41:50Z',
 'proof': {'type': 'Ed25519Signature2018',
  'proofPurpose': 'assertionMethod',
  'verificationMethod': 'did:key:z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD#z6MkkdLPuLhUz77kYQM6sRNSD9a5qqHW8WRcssSAWoquPRKD',
  'created': '2022-11-02T18:46:30.247Z',
  'jws': 'eyJhbGciOiJFZERTQSIsImNyaXQiOlsiYjY0Il0sImI2NCI6ZmFsc2V9..KvNTOJ8eL_0lZ_a8xcEOTXpMiqH6aaMz5tsRM5t3B7QrAT6Hq3sVELCy7RGicpgRhsdDippMLFZUrhVXWRAtDw'}}

In [41]:
!pip3 install aries-cloudagent

     |████████████████████████████████| 1.7 MB 2.0 MB/s eta 0:00:01
  Using cached PyJWT-2.4.0-py3-none-any.whl (18 kB)
     |████████████████████████████████| 40 kB 16.5 MB/s eta 0:00:01
  Using cached prompt_toolkit-2.0.10-py3-none-any.whl (340 kB)
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 1.7.1
    Uninstalling PyJWT-1.7.1:
      Successfully uninstalled PyJWT-1.7.1
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.31
    Uninstalling prompt-toolkit-3.0.31:
      Successfully uninstalled prompt-toolkit-3.0.31
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 8.5.0 requires prompt-toolkit<3.1.0,>3.0.1, but you have prompt-toolki

In [35]:
#print pip package version
package_version = 

'didkit'

In [174]:
from aries_cloudagent.wallet.key_type import KeyType
from aries_cloudagent.did.did_key import DIDKey
from aries_cloudagent import wallet
from aries_cloudagent.vc.ld_proofs import Ed25519Signature2018, JwsLinkedDataSignature
key = wallet.crypto.create_keypair(
    KeyType.ED25519
        )
didkey = DIDKey.from_public_key(key[0], key_type=KeyType.ED25519)
suite = Ed25519Signature2018(key_pair=key, verification_method=didkey.key_id)


test_cred = {
    "@context": [
        "https://www.w3.org/2018/credentials/v1",
        "https://www.w3.org/2018/credentials/examples/v1",
    ],
    "id": "http://example.gov/credentials/3732",
    "type": ["VerifiableCredential", "UniversityDegreeCredential"],
    "issuer": {"id": "did:key:z6Mkgg342Ycpuk263R9d8Aq6MUaxPn1DDeHyGo38EefXmgDL"},
    "issuanceDate": "2020-03-10T04:24:12.164Z",
    "credentialSubject": {
        "id": "did:example:456",
        "degree": {"type": "BachelorDegree", "name": "Bachelor of Science and Arts"},
    },
}
from aries_cloudagent.config.default_context import DefaultContextBuilder
from aries_cloudagent.vc.ld_proofs.document_loader import DocumentLoader
contextbl = DefaultContextBuilder()
context = await contextbl.build_context()

from aries_cloudagent.config.wallet import wallet_config
profile, _ = await wallet_config(context)
from aries_cloudagent.core.profile import Profile
dl = DocumentLoader(profile=profile)

from aries_cloudagent.vc.vc_ld.issue import issue
await issue(credential=test_cred, suite= suite, document_loader=dl)

AttributeError: 'tuple' object has no attribute 'sign'

In [194]:
import requests
vc_jff = requests.get("https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/jff-vc-edu-plugfest-1.json").content
vc_jff

b'<!DOCTYPE html>\n<html lang="en-US">\n  <head>\n    <meta charset="UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n\n<!-- Begin Jekyll SEO tag v2.8.0 -->\n<title>vc-ed | Verifiable Credentials for Education Task Force</title>\n<meta name="generator" content="Jekyll v3.9.2" />\n<meta property="og:title" content="vc-ed" />\n<meta property="og:locale" content="en_US" />\n<meta name="description" content="Verifiable Credentials for Education Task Force" />\n<meta property="og:description" content="Verifiable Credentials for Education Task Force" />\n<link rel="canonical" href="https://w3c-ccg.github.io/vc-ed/404.html" />\n<meta property="og:url" content="https://w3c-ccg.github.io/vc-ed/404.html" />\n<meta property="og:site_name" content="vc-ed" />\n<meta property="og:type" content="website" />\n<meta name="twitter:card" content="summary" />\n<meta property="twitter:title" content="vc-ed" />\n<sc

In [249]:
import didkit
jwk = didkit.generate_ed25519_key()
did = didkit.key_to_did("key", jwk)

credential = {
      "@context": 
      [
          "https://www.w3.org/2018/credentials/v1",
          {
    "id": "@id",
    "type": "@type",
    "xsd": "https://www.w3.org/2001/XMLSchema#",
    "OpenBadgeCredential": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#OpenBadgeCredential"
    },
    "Achievement": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Achievement",
      "@context": {
        "achievementType": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#achievementType",
          "@type": "xsd:string"
        },
        "alignment": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Alignment"
        },
        "creator": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Profile"
        },
        "creditsAvailable": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#creditsAvailable",
          "@type": "xsd:float"
        },
        "criteria": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#criteria", 
          "@type": "@id"
        },
        "fieldOfStudy": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#fieldOfStudy", 
          "@type": "xsd:string"
        },
        "humanCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#humanCode", 
          "@type": "xsd:string"
        },
        "specialization": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#specialization", 
          "@type": "xsd:string"
        },
        "tags": {
          "@id": "https://schema.org/keywords", 
          "@type": "xsd:string", 
          "@container": "@set"
        }
      }
    },
    "AchievementCredential": {
      "@id": "OpenBadgeCredential"
    },
    "AchievementSubject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#AchievementSubject",
      "@context": {
        "achievement": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Achievement"
        },
        "identifier": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Identifier"
        },
        "result": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Result"
        }
      }
    },
    "Address": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Address",
      "@context": {
        "addressCountry": {
          "@id": "https://schema.org/addressCountry", 
          "@type": "xsd:string"
        },
        "addressCountryCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#CountryCode", 
          "@type": "xsd:string"
        },
        "addressLocality": {
          "@id": "https://schema.org/addresLocality", 
          "@type": "xsd:string"
        },
        "addressRegion": {
          "@id": "https://schema.org/addressRegion", 
          "@type": "xsd:string"
        },
        "geo": {
          "@id" : "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#GeoCoordinates"
        },
        "postOfficeBoxNumber": {
          "@id": "https://schema.org/postOfficeBoxNumber", 
          "@type": "xsd:string"
        },
        "postalCode": {
          "@id": "https://schema.org/postalCode", 
          "@type": "xsd:string"
        },
        "streetAddress": {
          "@id": "https://schema.org/streetAddress", 
          "@type": "xsd:string"
        }
      }
    },
    "Alignment": {
      "@id": "https://schema.org/Alignment",
      "@context": {
        "targetCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#targetCode", 
          "@type": "xsd:string"
        },
        "targetDescription": {
          "@id": "https://schema.org/targetDescription", 
          "@type": "xsd:string"
        },
        "targetFramework": {
          "@id": "https://schema.org/targetFramework", 
          "@type": "xsd:string"
        },
        "targetName": {
          "@id": "https://schema.org/targetName", 
          "@type": "xsd:string"
        }, 
        "targetType": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#targetType",
          "@type": "xsd:string"
        },     
        "targetUrl": {
          "@id": "https://schema.org/targetUrl",
          "@type": "xsd:anyURI"
        }
      }
    },
    "Criteria": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Criteria"
    },
    "EndorsementCredential": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#EndorsementCredential"
    },
    "EndorsementSubject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#EndorsementSubject",
      "@context": {
        "endorsementComment": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#endorsementComment", 
          "@type": "xsd:string"
        }
      }
    },
    "Evidence": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Evidence",
      "@context": {
        "audience": {
          "@id": "https://schema.org/audience", 
          "@type": "xsd:string"
        },
        "genre": {
          "@id": "https://schema.org/genre",
          "@type": "xsd:string"
        }
      }
    },
    "GeoCoordinates": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#GeoCoordinates",
      "@context": {
        "latitude": {
          "@id": "https://schema.org/latitude", 
          "@type": "xsd:string"
        },
        "longitude": {
          "@id": "https://schema.org/longitude", 
          "@type": "xsd:string"
        }
      }
    },
    "IdentityObject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#IdentityObject",
      "@context": {
        "hashed": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#hashed", 
          "@type": "xsd:boolean"
        },
        "identityHash": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#identityHash", 
          "@type": "xsd:string"
        },
        "salt":  {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#salt", 
          "@type": "xsd:string"
        }
      }
    },
    "Image": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#IdentityImage",
      "@context": {
        "caption": {
          "@id": "https://schema.org/caption",
          "@type": "xsd:string"
        }
      }
    },
    "Profile": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Profile",
      "@context": {
        "additionalName": {
          "@id": "https://schema.org/additionalName",
          "@type": "xsd:string"
        },
        "address": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Address"
        },
        "dateOfBirth": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#dateOfBirth",
          "@type": "xsd:date"
        },
        "email": {
          "@id": "https://schema.org/email", 
          "@type": "xsd:string"
        },
        "familyName": {
          "@id": "https://schema.org/familyName",
          "@type": "xsd:string"
        },
        "familyNamePrefix": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#familyNamePrefix",
          "@type": "xsd:string"
        },
        "givenName": {
          "@id": "https://schema.org/givenName",
          "@type": "xsd:string"
        },
        "honorificPrefix": {
          "@id": "https://schema.org/honorificPrefix",
          "@type": "xsd:string"
        },
        "honorificSuffix": {
          "@id": "https://schema.org/honorificSuffix",
          "@type": "xsd:string"
        },
        "parentOrg": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#parentOrg",
          "@type": "xsd:string"
        },
        "patronymicName": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#patronymicName",
          "@type": "xsd:string"
        },
        "phone": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#PhoneNumber", 
          "@type": "xsd:string"
        },
        "official": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#official",
          "@type": "xsd:string"
        },
        "sisSourcedId": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#sisSourcedId",
          "@type": "xsd:string"
        },
        "sourcedId": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#sourcedId",
          "@type": "xsd:string"
        }
      }
    },
    "Result": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Result",
      "@context": {
        "achievedLevel": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#achievedLevel", 
          "@type": "xsd:anyURI"
        },
        "resultDescription": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#resultDescription",
          "@type": "xsd:anyURI"
        },
        "status": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#status", 
          "@type": "xsd:string"
        },
        "value": {
          "@id": "https://schema.org/value", 
          "@type": "xsd:string"
        }
      }
    },
    "ResultDescription": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#ResultDescription",
      "@context": {
        "allowedValue": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#allowedValue", 
          "@type": "xsd:string"
        },
        "requiredLevel": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#requiredLevel", 
          "@type": "xsd:anyURI"
        },
        "requiredValue": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#requiredValue", 
          "@type": "xsd:string"
        },
        "resultType": {
          "@id":"https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#resultType",  
          "@type": "xsd:string"
        },
        "rubricCriterionLevel": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#RubricCriterionLevel",
        "valueMax": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#valueMax", 
          "@type": "xsd:string"
        },
        "valueMin": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#valueMin", 
          "@type": "xsd:string"
        }              
      }
    },
    "RubricCriterionLevel": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#RubricCriterionLevel",
      "@context": {
        "level": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#level", 
          "@type": "xsd:string"
        },
        "points": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#points", 
          "@type": "xsd:string"
        }
      }
    },
    "alignment": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Alignment", 
      "@type": "@id"
    },    
    "description": {
      "@id": "https://schema.org/description", 
      "@type": "xsd:string"
    },
    "endorsement": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#endorsement", 
      "@type": "@id"
    },
    "image": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#image", 
      "@type": "@id"
    },
    "name": {
      "@id": "https://schema.org/name", 
      "@type": "xsd:string"
    },
    "narrative": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#narrative", 
      "@type": "xsd:string"
    },
    "url": {
      "@id": "https://schema.org/url", 
      "@type": "xsd:anyURI"
    }
  }
      ],
      "id": str(uuid.uuid4()),
      "type": ["VerifiableCredential", "OpenBadgeCredential"],
      "issuer":  did,
      "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "credentialSubject": {
          "type": "AchievementSubject",
          "id": 'holder_did',
          "achievement": {
            "type": "Achievement",
            "name": "IIW 2022 DEMO",
            "description": "This wallet can display this Open Badge 3.0",
            "criteria": {
              "type": "Criteria",
              "narrative": "The first cohort of the JFF Plugfest 1 in May/June of 2021 collaborated to push interoperability of VCs in education forward."
            },
            "image": "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/images/plugfest-1-badge-image.png"
          }
          }
      #   },
      # "options": {
      #     "proofType": "Ed25519Signature2018"
      # }
}
test_cred = {
    "@context": [
        "https://www.w3.org/2018/credentials/v1",
        {
    "id": "@id",
    "type": "@type",
    "xsd": "https://www.w3.org/2001/XMLSchema#",
    "OpenBadgeCredential": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#OpenBadgeCredential"
    },
    "Achievement": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Achievement",
      "@context": {
        "achievementType": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#achievementType",
          "@type": "xsd:string"
        },
        "alignment": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Alignment"
        },
        "creator": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Profile"
        },
        "creditsAvailable": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#creditsAvailable",
          "@type": "xsd:float"
        },
        "criteria": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#criteria", 
          "@type": "@id"
        },
        "fieldOfStudy": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#fieldOfStudy", 
          "@type": "xsd:string"
        },
        "humanCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#humanCode", 
          "@type": "xsd:string"
        },
        "specialization": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#specialization", 
          "@type": "xsd:string"
        },
        "tags": {
          "@id": "https://schema.org/keywords", 
          "@type": "xsd:string", 
          "@container": "@set"
        }
      }
    },
    "AchievementCredential": {
      "@id": "OpenBadgeCredential"
    },
    "AchievementSubject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#AchievementSubject",
      "@context": {
        "achievement": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Achievement"
        },
        "identifier": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Identifier"
        },
        "result": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Result"
        }
      }
    },
    "Address": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Address",
      "@context": {
        "addressCountry": {
          "@id": "https://schema.org/addressCountry", 
          "@type": "xsd:string"
        },
        "addressCountryCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#CountryCode", 
          "@type": "xsd:string"
        },
        "addressLocality": {
          "@id": "https://schema.org/addresLocality", 
          "@type": "xsd:string"
        },
        "addressRegion": {
          "@id": "https://schema.org/addressRegion", 
          "@type": "xsd:string"
        },
        "geo": {
          "@id" : "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#GeoCoordinates"
        },
        "postOfficeBoxNumber": {
          "@id": "https://schema.org/postOfficeBoxNumber", 
          "@type": "xsd:string"
        },
        "postalCode": {
          "@id": "https://schema.org/postalCode", 
          "@type": "xsd:string"
        },
        "streetAddress": {
          "@id": "https://schema.org/streetAddress", 
          "@type": "xsd:string"
        }
      }
    },
    "Alignment": {
      "@id": "https://schema.org/Alignment",
      "@context": {
        "targetCode": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#targetCode", 
          "@type": "xsd:string"
        },
        "targetDescription": {
          "@id": "https://schema.org/targetDescription", 
          "@type": "xsd:string"
        },
        "targetFramework": {
          "@id": "https://schema.org/targetFramework", 
          "@type": "xsd:string"
        },
        "targetName": {
          "@id": "https://schema.org/targetName", 
          "@type": "xsd:string"
        }, 
        "targetType": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#targetType",
          "@type": "xsd:string"
        },     
        "targetUrl": {
          "@id": "https://schema.org/targetUrl",
          "@type": "xsd:anyURI"
        }
      }
    },
    "Criteria": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Criteria"
    },
    "EndorsementCredential": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#EndorsementCredential"
    },
    "EndorsementSubject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#EndorsementSubject",
      "@context": {
        "endorsementComment": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#endorsementComment", 
          "@type": "xsd:string"
        }
      }
    },
    "Evidence": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Evidence",
      "@context": {
        "audience": {
          "@id": "https://schema.org/audience", 
          "@type": "xsd:string"
        },
        "genre": {
          "@id": "https://schema.org/genre",
          "@type": "xsd:string"
        }
      }
    },
    "GeoCoordinates": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#GeoCoordinates",
      "@context": {
        "latitude": {
          "@id": "https://schema.org/latitude", 
          "@type": "xsd:string"
        },
        "longitude": {
          "@id": "https://schema.org/longitude", 
          "@type": "xsd:string"
        }
      }
    },
    "IdentityObject": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#IdentityObject",
      "@context": {
        "hashed": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#hashed", 
          "@type": "xsd:boolean"
        },
        "identityHash": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#identityHash", 
          "@type": "xsd:string"
        },
        "salt":  {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#salt", 
          "@type": "xsd:string"
        }
      }
    },
    "Image": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#IdentityImage",
      "@context": {
        "caption": {
          "@id": "https://schema.org/caption",
          "@type": "xsd:string"
        }
      }
    },
    "Profile": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Profile",
      "@context": {
        "additionalName": {
          "@id": "https://schema.org/additionalName",
          "@type": "xsd:string"
        },
        "address": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Address"
        },
        "dateOfBirth": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#dateOfBirth",
          "@type": "xsd:date"
        },
        "email": {
          "@id": "https://schema.org/email", 
          "@type": "xsd:string"
        },
        "familyName": {
          "@id": "https://schema.org/familyName",
          "@type": "xsd:string"
        },
        "familyNamePrefix": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#familyNamePrefix",
          "@type": "xsd:string"
        },
        "givenName": {
          "@id": "https://schema.org/givenName",
          "@type": "xsd:string"
        },
        "honorificPrefix": {
          "@id": "https://schema.org/honorificPrefix",
          "@type": "xsd:string"
        },
        "honorificSuffix": {
          "@id": "https://schema.org/honorificSuffix",
          "@type": "xsd:string"
        },
        "parentOrg": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#parentOrg",
          "@type": "xsd:string"
        },
        "patronymicName": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#patronymicName",
          "@type": "xsd:string"
        },
        "phone": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#PhoneNumber", 
          "@type": "xsd:string"
        },
        "official": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#official",
          "@type": "xsd:string"
        },
        "sisSourcedId": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#sisSourcedId",
          "@type": "xsd:string"
        },
        "sourcedId": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#sourcedId",
          "@type": "xsd:string"
        }
      }
    },
    "Result": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Result",
      "@context": {
        "achievedLevel": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#achievedLevel", 
          "@type": "xsd:anyURI"
        },
        "resultDescription": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#resultDescription",
          "@type": "xsd:anyURI"
        },
        "status": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#status", 
          "@type": "xsd:string"
        },
        "value": {
          "@id": "https://schema.org/value", 
          "@type": "xsd:string"
        }
      }
    },
    "ResultDescription": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#ResultDescription",
      "@context": {
        "allowedValue": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#allowedValue", 
          "@type": "xsd:string"
        },
        "requiredLevel": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#requiredLevel", 
          "@type": "xsd:anyURI"
        },
        "requiredValue": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#requiredValue", 
          "@type": "xsd:string"
        },
        "resultType": {
          "@id":"https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#resultType",  
          "@type": "xsd:string"
        },
        "rubricCriterionLevel": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#RubricCriterionLevel",
        "valueMax": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#valueMax", 
          "@type": "xsd:string"
        },
        "valueMin": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#valueMin", 
          "@type": "xsd:string"
        }              
      }
    },
    "RubricCriterionLevel": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#RubricCriterionLevel",
      "@context": {
        "level": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#level", 
          "@type": "xsd:string"
        },
        "points": {
          "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#points", 
          "@type": "xsd:string"
        }
      }
    },
    "alignment": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Alignment", 
      "@type": "@id"
    },    
    "description": {
      "@id": "https://schema.org/description", 
      "@type": "xsd:string"
    },
    "endorsement": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#endorsement", 
      "@type": "@id"
    },
    "image": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#image", 
      "@type": "@id"
    },
    "name": {
      "@id": "https://schema.org/name", 
      "@type": "xsd:string"
    },
    "narrative": {
      "@id": "https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#narrative", 
      "@type": "xsd:string"
    },
    "url": {
      "@id": "https://schema.org/url", 
      "@type": "xsd:anyURI"
    }
  }
      
    ],
    "id": "http://example:123456789abcdefghi",
    "type": ["VerifiableCredential", "OpenBadgeCredential"],
    "issuer": {"id":did},
    "issuanceDate": "2020-03-10T04:24:12.164Z",
    "credentialSubject": {
          "type": "VerifiableCredential",
          "id": "did:key:123456789abcdefghi",
          "Achievement": {
            "type": "Achievement",
            "name": "IIW 2022 DEMO",
            "description": "This wallet can display this Open Badge 3.0",
            "criteria": {
              "type": "Criteria",
              "narrative": "The first cohort of the JFF Plugfest 1 in May/June of 2021 collaborated to push interoperability of VCs in education forward."
            },
            "image": "https://w3c-ccg.github.io/vc-ed/plugfest-1-2022/images/plugfest-1-badge-image.png"
          }
# "degree": {"type": "BachelorDegree", "name": "Bachelor of Science and Arts"},
      
        }
}
# test_cred = {
#     "@context": [
#         "https://www.w3.org/2018/credentials/v1",
#         "https://www.w3.org/2018/credentials/examples/v1",
#     ],
#     "id": "http://example.gov/credentials/3732",
#     "type": ["VerifiableCredential", "UniversityDegreeCredential"],
#     "issuer": {"id": did},
#     "issuanceDate": "2020-03-10T04:24:12.164Z",
#     "credentialSubject": {
#         "id": "did:example:456",
#         "degree": {"type": "BachelorDegree", "name": "Bachelor of Science and Arts"},
#     },
# }
signed_credential = await didkit.issue_credential(
        json.dumps(test_cred),
        '{}',
        jwk)
print(json.loads(signed_credential))

{'@context': ['https://www.w3.org/2018/credentials/v1', {'xsd': 'https://www.w3.org/2001/XMLSchema#', 'url': {'@id': 'https://schema.org/url', '@type': 'xsd:anyURI'}, 'AchievementSubject': {'@context': {'achievement': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Achievement'}, 'identifier': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Identifier'}, 'result': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#Result'}}, '@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#AchievementSubject'}, 'IdentityObject': {'@context': {'hashed': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#hashed', '@type': 'xsd:boolean'}, 'identityHash': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#identityHash', '@type': 'xsd:string'}, 'salt': {'@id': 'https://imsglobal.github.io/openbadges-specification/ob_v3p0.html#salt', '@type': 'xsd:string'}}, '

In [189]:
import asyncio
import didkit
import json

jwk = didkit.generate_ed25519_key()
did = didkit.key_to_did("key", jwk)
print('jwk:', jwk)
print('did:', did)
challenge = await didkit.did_auth(did,'{"challenge":"alex", "domain":"alexa"}',jwk)
# await didkit.did_auth(did,'{\'challenge\':\'alex-asxe1341\'}',jwk)

jwk: {"kty":"OKP","crv":"Ed25519","x":"mEPYN-fZEcfawej4ASQ9Mt1WiTKYCC42oX1pYsWBYHQ","d":"pNLI1pN-ZdvlqfYshYYFsz2fdY3yyHx9dgJS4lc0Pmc"}
did: did:key:z6Mkphe1ydPursVEEoMFJNuPcGzCi2u12AEAK2vwcxikk2EP


In [146]:
await didkit.verify_presentation(challenge,'{"challenge":"alex", "domain":"alexa"}')

'{"checks":["proof"],"warnings":[],"errors":[]}'

In [147]:
holder_did

'did:peer:2.Ez6LSff5cxwvPXYvfP9KxtXx3tuikxBt9yRzZcfWKskPFyzEs.Vz6MkmFC5xTHkGEnjx9sK8Pz1fz5kvwjyrsV9QmaVKcNXQSFr.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0'

In [51]:
signed_credential = await didkit.issue_credential(
        json.dumps(credential_request['credential']),
        json.dumps({}),
        jwk)
print(json.loads(signed_credential))

ValueError: Invalid URI at line 1 column 187

In [197]:
res = await didkit.verify_credential(signed_credential, json.dumps({}))

'{"checks":["proof"],"warnings":[],"errors":[]}'

In [201]:
await didkit.verify_credential(json.dumps(test_cred), json.dumps({}))

'{"checks":[],"warnings":[],"errors":["No applicable proof"]}'

'{"checks":["proof"],"warnings":[],"errors":["signature error: Verification equation was not satisfied"]}'

In [204]:
signed_credential

'{"@context":["https://www.w3.org/2018/credentials/v1","https://www.w3.org/2018/credentials/examples/v1"],"id":"http://example.gov/credentials/3732","type":["VerifiableCredential","UniversityDegreeCredential"],"credentialSubject":{"id":"did:example:456","degree":{"name":"Bachelor of Science and Arts","type":"BachelorDegree"}},"issuer":"did:key:z6MkpXFs2MFWcdVimqZ8GbiP6UpwhdfsaPrNp45YxuZ8TfWo","issuanceDate":"2020-03-10T04:24:12.164Z","proof":{"type":"Ed25519Signature2018","proofPurpose":"assertionMethod","verificationMethod":"did:key:z6MkpXFs2MFWcdVimqZ8GbiP6UpwhdfsaPrNp45YxuZ8TfWo#z6MkpXFs2MFWcdVimqZ8GbiP6UpwhdfsaPrNp45YxuZ8TfWo","created":"2022-11-02T17:38:41.879Z","jws":"eyJhbGciOiJFZERTQSIsImNyaXQiOlsiYjY0Il0sImI2NCI6ZmFsc2V9..VzRMl3ea9Ra1zkqNwaQKxY0wYjggUVjBnZnOCBdhA4vebh2IYnFhl9aQDdANtqQJkxxJgLCmFCOXjt1dRcf9Dw"}}'

In [265]:
print(didkit.__dict__)

{'__name__': 'didkit', '__doc__': '', '__package__': 'didkit', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x1279d4400>, '__spec__': ModuleSpec(name='didkit', loader=<_frozen_importlib_external.SourceFileLoader object at 0x1279d4400>, origin='/usr/local/lib/python3.8/site-packages/didkit/__init__.py', submodule_search_locations=['/usr/local/lib/python3.8/site-packages/didkit']), '__path__': ['/usr/local/lib/python3.8/site-packages/didkit'], '__file__': '/usr/local/lib/python3.8/site-packages/didkit/__init__.py', '__cached__': '/usr/local/lib/python3.8/site-packages/didkit/__pycache__/__init__.cpython-38.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>), '__